Necessary Imports

In [ ]:
import base64
import requests
import os

Setting OpenAI API Key, better be done by setting an environment variable

In [ ]:
api_key = "your-openai-api-key"

Function to encode images using base 64

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

Path to directory with CAM images stored

In [ ]:
directory_path = "/path/to/directory"

Setting API Headers

In [ ]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

Iterating through all files in the directory to 1)Encode each image 2)Feed it to the LLM via the API 3)Get a response(description) for each image 

In [ ]:
for filename in os.listdir(directory_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        image_path = os.path.join(directory_path, filename)
        
        # Encode the image
        base64_image = encode_image(image_path)
        
        # Prepare the payload
        payload = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "You are a helpful assistant. Describe what you can see in this image, paying attention to key concepts and relationship between them"
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        
        # Send the request to the OpenAI API
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        
        # Print the response
        print(f"Response for {filename}:")
        print(response.json()['choices'][0]['message']['content'])
        print("\n" + "="*50 + "\n")